# Import / Setup

In [1]:
!pip install --upgrade pandas
import pandas as pd
import csv
import numpy as np
!pip install essentia-tensorflow
import essentia.standard as es
import IPython.display as ipd
import json
from pathlib import Path
import time
import ast


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 MB 5.1 MB/s eta 0:00:00


In [2]:
# /!\ PATH MANAGEMENT HERE /!\

!export my_home=/content/gdrive/MyDrive/Term2/ASMLab/Part2
!export my_home_mp3_subfolder=$my_home/audio_chunks/
my_home = '/content/gdrive/MyDrive/Term2/ASMLab/Part2'

In [ ]:
!ls $my_home_mp3_subfolder

audio.000  audio.001  audio.002  audio.003  audio.004  audio.005  audio.006


In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd $my_home_mp3_subfolder
%ls -lh

/content/gdrive/.shortcut-targets-by-id/1c5yJMw7znSoohe7ad1SoTKTvJlXS4Rux/audio_chunks
total 28K
dr-x------ 230 root root 4.0K Jan 25 14:03 audio.000/
dr-x------ 231 root root 4.0K Jan 25 14:03 audio.001/
dr-x------ 224 root root 4.0K Jan 25 14:03 audio.002/
dr-x------ 228 root root 4.0K Jan 25 14:03 audio.003/
dr-x------ 216 root root 4.0K Jan 25 14:03 audio.004/
dr-x------ 231 root root 4.0K Jan 25 14:03 audio.005/
dr-x------ 222 root root 4.0K Jan 25 14:03 audio.006/


In [ ]:
# We write all the paths of mp3 files to a csv file
!find $my_home_mp3_subfolder -name '*.mp3' >  $my_home/pathlist.csv

In [ ]:
!wc -l $my_home/pathlist.csv

2100 /content/gdrive/MyDrive/Term2/ASMLab/Part2//pathlist.csv


In [4]:
audiopath = []
with open(f'{my_home}/pathlist.csv') as fp:
    csvreader = csv.reader(fp)
    for line in csvreader:
        audiopath.append(line[0])
print(audiopath)
ipd.Audio(filename=audiopath[1])

['/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/5Z/5Z54RgCfhRljLVjPZHy5dv.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/7G/7GgYmXY3PfDjTiyjUlvF7Y.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/0z/0zf1BQJ4om2qU0W9muvnLn.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/4s/4sCcDvX30uu39ozvxcRsqB.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/6Z/6ZBiXweylRlROqwP3ODYgw.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/4X/4XLVg8qERMwZBy0HHr2DdW.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/4X/4XQETu7QYGbUhTt68kekOn.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/4r/4rhaDfV9y5UvGZ67BT8U35.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/2D/2DOfw6UDf0eyCdb0SMVZrE.mp3', '/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/5f/5fz5MpkCMxmkwmnPCelvaK.mp3',

# Preparation the models

In [ ]:
# Curling the *.pb files
!curl -o $my_home/discogs-effnet-bs64-1.pb https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
!curl -o $my_home/voice_instrumental-musicnn-msd-1.pb https://essentia.upf.edu/models/classifiers/voice_instrumental/voice_instrumental-musicnn-msd-1.pb
!curl -o $my_home/msd-musicnn-1.pb https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb
!curl -o $my_home/emomusic-musicnn-msd-2.pb https://essentia.upf.edu/models/classification-heads/emomusic/emomusic-musicnn-msd-2.pb

!curl -o $my_home/discogs-effnet-bs64-1.json https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.5M  100 17.5M    0     0   554k      0  0:00:32  0:00:32 --:--:--  526k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3163k  100 3163k    0     0   484k      0  0:00:06  0:00:06 --:--:--  659k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3123k  100 3123k    0     0   534k      0  0:00:05  0:00:05 --:--:--  762k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 82460  100 82460    0     0  53825      0  0:00:01  0:00:01 --:--:-- 53860
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
# Music style classifier
model_musicstyle = es.TensorflowPredictEffnetDiscogs(graphFilename=f"{my_home}/discogs-effnet-bs64-1.pb")

# Voice / Instru classifier
model_voiceinstru = es.TensorflowPredictMusiCNN(graphFilename=f'{my_home}/voice_instrumental-musicnn-msd-1.pb')

# Arousal / Valence
embeddings = es.TensorflowPredictMusiCNN(graphFilename=f'{my_home}/msd-musicnn-1.pb', output = 'model/dense/BiasAdd')
emotion = es.TensorflowPredict2D(graphFilename=f'{my_home}/emomusic-musicnn-msd-2.pb', output='model/Identity')


# Extraction the data and putting it to a csv and a pickle



In [ ]:
f = open(f'{my_home}/discogs-effnet-bs64-1.json')
data = json.load(f)
classes = data['classes']
print(classes)
f.close()

['Blues---Boogie Woogie', 'Blues---Chicago Blues', 'Blues---Country Blues', 'Blues---Delta Blues', 'Blues---Electric Blues', 'Blues---Harmonica Blues', 'Blues---Jump Blues', 'Blues---Louisiana Blues', 'Blues---Modern Electric Blues', 'Blues---Piano Blues', 'Blues---Rhythm & Blues', 'Blues---Texas Blues', 'Brass & Military---Brass Band', 'Brass & Military---Marches', 'Brass & Military---Military', "Children's---Educational", "Children's---Nursery Rhymes", "Children's---Story", 'Classical---Baroque', 'Classical---Choral', 'Classical---Classical', 'Classical---Contemporary', 'Classical---Impressionist', 'Classical---Medieval', 'Classical---Modern', 'Classical---Neo-Classical', 'Classical---Neo-Romantic', 'Classical---Opera', 'Classical---Post-Modern', 'Classical---Renaissance', 'Classical---Romantic', 'Electronic---Abstract', 'Electronic---Acid', 'Electronic---Acid House', 'Electronic---Acid Jazz', 'Electronic---Ambient', 'Electronic---Bassline', 'Electronic---Beatdown', 'Electronic---Ber

In [ ]:
# Verification that audiopath works.
print(audiopath[1])
tmp = Path(audiopath[1])
p = tmp.name
print(p)

/content/gdrive/MyDrive/Term2/ASMLab/Part2//audio_chunks/audio.001/7G/7GgYmXY3PfDjTiyjUlvF7Y.mp3
7GgYmXY3PfDjTiyjUlvF7Y.mp3


In [ ]:
iter = 0
error = 0
nb_file = len(audiopath)
start = time.monotonic()
with open(f'{my_home}/feature_extracted_2.csv', 'w') as fp:
  writer = csv.writer(fp)
  writer.writerow(['fullpath', 'filename', 'bpm', 'danceability', 'voice', 'instru', 'arousal', 'valence', 'music_style'])
  #writer.writerows( a list of lists, writing each one on a new line )

  for audio_file in audiopath:
    iter += 1
    if(iter % 20 == 0):
      print(f'{iter} files processed on {nb_file} - {iter/nb_file*100} %')
      print(f'Errors so far : {error} ; error rate : {error/iter*100}')
    audio_path = Path(audio_file)
    full_path = audio_path.relative_to(f'{my_home}/')
    filename = audio_path.name
    audio = es.MonoLoader(filename=audio_file)()

    try:
        # BPM 
        rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
        bpm, _, conf, _, _ = rhythm_extractor(audio)
        #print("BPM:", bpm)

        # Danceability
        danceability = es.Danceability()(audio)
        #print('Danceability score', danceability[0])

        # Voice Instru classif
        activations_voiceinstru = model_voiceinstru(audio)
        mean_voiceinstru = np.mean(activations_voiceinstru, axis=0)
        #print('Mean :',mean_voiceinstru)  

        # Arousal / Valence
        embeds = embeddings(audio)
        activations_arousal_valence = emotion(embeds)
        mean_arousal_valence = np.mean(activations_arousal_valence, axis=0)
        #print('Mean :',mean_arousal_valence)

        # Music Style
        activations_musicstyle = model_musicstyle(audio)
        mean_musicstyle = np.mean(activations_musicstyle, axis=0)
        #print('Mean :',mean_musicstyle)
        dico = {}
        for i in range(len(mean_musicstyle)):
          dico[classes[i]] = mean_musicstyle[i]

        # filename', 'bpm', 'danceability', 'voice', 'instru', 'arousal', 'valence', 'music_style'
        list_csv = [full_path, filename, bpm, danceability[0], mean_voiceinstru[0], mean_voiceinstru[1], mean_arousal_valence[0], mean_arousal_valence[1], dico]
        writer.writerow(list_csv)
    
    except:
      error += 1

end = time.monotonic()
total = end - start
print(f'End of processing - it took {total} seconds')
print(f'Error : {error} ; error rate : {error/nb_file*100}')

20 files processed on 1320 - 1.5151515151515151 %
Errors so far : 1 ; error rate : 5.0
40 files processed on 1320 - 3.0303030303030303 %
Errors so far : 1 ; error rate : 2.5
60 files processed on 1320 - 4.545454545454546 %
Errors so far : 1 ; error rate : 1.6666666666666667
80 files processed on 1320 - 6.0606060606060606 %
Errors so far : 1 ; error rate : 1.25
100 files processed on 1320 - 7.575757575757576 %
Errors so far : 1 ; error rate : 1.0
120 files processed on 1320 - 9.090909090909092 %
Errors so far : 1 ; error rate : 0.8333333333333334
140 files processed on 1320 - 10.606060606060606 %
Errors so far : 1 ; error rate : 0.7142857142857143
160 files processed on 1320 - 12.121212121212121 %
Errors so far : 1 ; error rate : 0.625
180 files processed on 1320 - 13.636363636363635 %
Errors so far : 1 ; error rate : 0.5555555555555556
200 files processed on 1320 - 15.151515151515152 %
Errors so far : 1 ; error rate : 0.5
220 files processed on 1320 - 16.666666666666664 %
Errors so far

In [ ]:
list_column = ['fullpath', 'filename', 'bpm', 'danceability', 'voice', 'instru', 'arousal', 'valence']
[list_column.append(i) for i in classes]
print(list_column)

['fullpath', 'filename', 'bpm', 'danceability', 'voice', 'instru', 'arousal', 'valence', 'Blues---Boogie Woogie', 'Blues---Chicago Blues', 'Blues---Country Blues', 'Blues---Delta Blues', 'Blues---Electric Blues', 'Blues---Harmonica Blues', 'Blues---Jump Blues', 'Blues---Louisiana Blues', 'Blues---Modern Electric Blues', 'Blues---Piano Blues', 'Blues---Rhythm & Blues', 'Blues---Texas Blues', 'Brass & Military---Brass Band', 'Brass & Military---Marches', 'Brass & Military---Military', "Children's---Educational", "Children's---Nursery Rhymes", "Children's---Story", 'Classical---Baroque', 'Classical---Choral', 'Classical---Classical', 'Classical---Contemporary', 'Classical---Impressionist', 'Classical---Medieval', 'Classical---Modern', 'Classical---Neo-Classical', 'Classical---Neo-Romantic', 'Classical---Opera', 'Classical---Post-Modern', 'Classical---Renaissance', 'Classical---Romantic', 'Electronic---Abstract', 'Electronic---Acid', 'Electronic---Acid House', 'Electronic---Acid Jazz', 'El

In [ ]:
import ast 
with open(f'{my_home}/feature_extracted_final.csv') as fp:
  with open(f'{my_home}/feature_extracted_final_better_formated.csv', 'w') as fpw:
    csvreader = csv.reader(fp)
    writer = csv.writer(fpw)
    writer.writerow(list_column)
    dict_to_arr = {}
    for line in csvreader:
      if line[0] != 'fullpath':
        line_to_write = line[:-1]
        dict_to_arr = ast.literal_eval(line[-1])
        [line_to_write.append(i) for i in dict_to_arr.values()]
        writer.writerow(line_to_write)


In [7]:
# Writing to a pickle file :
import pickle
import pandas as pd
df = pd.read_csv(f'{my_home}/feature_extracted_final_better_formated.csv')
df.to_pickle(f'{my_home}/feature_extracted_final_better_formated.pickle')


In [ ]:
df

,fullpath,filename,bpm,danceability,voice,instru,arousal,valence,Blues---Boogie Woogie,Blues---Chicago Blues,...,Rock---Symphonic Rock,Rock---Technical Death Metal,Rock---Thrash,Rock---Twist,Rock---Viking Metal,Rock---Yé-Yé,Stage & Screen---Musical,Stage & Screen---Score,Stage & Screen---Soundtrack,Stage & Screen---Theme
0,audio_chunks/audio.001/5Z/5Z54RgCfhRljLVjPZHy5...,5Z54RgCfhRljLVjPZHy5dv.mp3,119.751747,1.027763,0.483431,0.494761,3.340118,3.374878,5.059277e-05,7.002735e-05,...,0.003889,0.002356,0.003100,2.519490e-04,0.010147,1.839559e-04,0.002199,0.015117,0.040985,0.008835
1,audio_chunks/audio.001/7G/7GgYmXY3PfDjTiyjUlvF...,7GgYmXY3PfDjTiyjUlvF7Y.mp3,119.751747,0.972829,0.334506,0.634316,3.489979,4.115166,6.216439e-04,3.023574e-04,...,0.003693,0.001057,0.006044,6.817841e-04,0.000954,6.278483e-04,0.000958,0.001658,0.010207,0.001044
2,audio_chunks/audio.001/0z/0zf1BQJ4om2qU0W9muvn...,0zf1BQJ4om2qU0W9muvnLn.mp3,119.751747,1.125426,0.573570,0.399495,3.702826,4.368959,4.495818e-05,1.415929e-04,...,0.004680,0.001475,0.009901,1.548242e-04,0.004939,1.047766e-04,0.000499,0.001569,0.006819,0.001377
3,audio_chunks/audio.001/4s/4sCcDvX30uu39ozvxcRs...,4sCcDvX30uu39ozvxcRsqB.mp3,119.751747,1.537965,0.261856,0.741760,3.040704,5.242820,9.991652e-06,4.228707e-04,...,0.000602,0.000092,0.001113,3.521542e-04,0.000497,9.158996e-05,0.000349,0.001024,0.003258,0.000436
4,audio_chunks/audio.001/6Z/6ZBiXweylRlROqwP3ODY...,6ZBiXweylRlROqwP3ODYgw.mp3,119.751747,1.151880,0.659588,0.325184,3.896225,4.917584,1.622540e-05,1.809556e-04,...,0.001226,0.000019,0.000578,3.791122e-05,0.000663,4.482077e-05,0.000276,0.001343,0.004523,0.001376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,audio_chunks/audio.002/3I/3I6v5wmP0joU1tbEDnBc...,3I6v5wmP0joU1tbEDnBcrt.mp3,92.253181,0.865726,0.575876,0.399688,3.290955,3.507015,8.601876e-06,1.169389e-05,...,0.004191,0.002211,0.002001,2.245610e-05,0.005744,2.874793e-05,0.001226,0.016970,0.044262,0.005239
2094,audio_chunks/audio.002/1x/1xM6rthhqPRmHfesVPjd...,1xM6rthhqPRmHfesVPjdCn.mp3,92.253181,1.218599,0.642027,0.332648,3.054225,4.300513,4.660618e-05,6.681142e-05,...,0.001146,0.000141,0.001809,1.310748e-04,0.000676,1.345923e-04,0.000583,0.002017,0.005453,0.001877
2095,audio_chunks/audio.002/1x/1xafrgeBMaBBrnr1e7rk...,1xafrgeBMaBBrnr1e7rkaC.mp3,92.253181,1.853582,0.684296,0.291994,3.000327,4.777125,3.826593e-06,2.911063e-05,...,0.000104,0.000022,0.000178,5.546825e-06,0.000298,1.237812e-05,0.000003,0.000169,0.000718,0.000085
2096,audio_chunks/audio.002/1M/1MTFN8gHszESSnKqXEgh...,1MTFN8gHszESSnKqXEghob.mp3,92.253181,2.013886,0.419342,0.548770,3.728135,3.764989,2.620683e-05,3.597971e-05,...,0.000417,0.000064,0.000605,1.277960e-04,0.000173,1.123576e-04,0.003637,0.001390,0.043248,0.002857


In [ ]:
df1 = pd.read_pickle(f'{my_home}/feature_extracted_final_better_formated.pickle')

In [ ]:
df1

,fullpath,filename,bpm,danceability,voice,instru,arousal,valence,Blues---Boogie Woogie,Blues---Chicago Blues,...,Rock---Symphonic Rock,Rock---Technical Death Metal,Rock---Thrash,Rock---Twist,Rock---Viking Metal,Rock---Yé-Yé,Stage & Screen---Musical,Stage & Screen---Score,Stage & Screen---Soundtrack,Stage & Screen---Theme
0,audio_chunks/audio.001/5Z/5Z54RgCfhRljLVjPZHy5...,5Z54RgCfhRljLVjPZHy5dv.mp3,119.751747,1.027763,0.483431,0.494761,3.340118,3.374878,5.059277e-05,7.002735e-05,...,0.003889,0.002356,0.003100,2.519490e-04,0.010147,1.839559e-04,0.002199,0.015117,0.040985,0.008835
1,audio_chunks/audio.001/7G/7GgYmXY3PfDjTiyjUlvF...,7GgYmXY3PfDjTiyjUlvF7Y.mp3,119.751747,0.972829,0.334506,0.634316,3.489979,4.115166,6.216439e-04,3.023574e-04,...,0.003693,0.001057,0.006044,6.817841e-04,0.000954,6.278483e-04,0.000958,0.001658,0.010207,0.001044
2,audio_chunks/audio.001/0z/0zf1BQJ4om2qU0W9muvn...,0zf1BQJ4om2qU0W9muvnLn.mp3,119.751747,1.125426,0.573570,0.399495,3.702826,4.368959,4.495818e-05,1.415929e-04,...,0.004680,0.001475,0.009901,1.548242e-04,0.004939,1.047766e-04,0.000499,0.001569,0.006819,0.001377
3,audio_chunks/audio.001/4s/4sCcDvX30uu39ozvxcRs...,4sCcDvX30uu39ozvxcRsqB.mp3,119.751747,1.537965,0.261856,0.741760,3.040704,5.242820,9.991652e-06,4.228707e-04,...,0.000602,0.000092,0.001113,3.521542e-04,0.000497,9.158996e-05,0.000349,0.001024,0.003258,0.000436
4,audio_chunks/audio.001/6Z/6ZBiXweylRlROqwP3ODY...,6ZBiXweylRlROqwP3ODYgw.mp3,119.751747,1.151880,0.659588,0.325184,3.896225,4.917584,1.622540e-05,1.809556e-04,...,0.001226,0.000019,0.000578,3.791122e-05,0.000663,4.482077e-05,0.000276,0.001343,0.004523,0.001376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,audio_chunks/audio.002/3I/3I6v5wmP0joU1tbEDnBc...,3I6v5wmP0joU1tbEDnBcrt.mp3,92.253181,0.865726,0.575876,0.399688,3.290955,3.507015,8.601876e-06,1.169389e-05,...,0.004191,0.002211,0.002001,2.245610e-05,0.005744,2.874793e-05,0.001226,0.016970,0.044262,0.005239
2094,audio_chunks/audio.002/1x/1xM6rthhqPRmHfesVPjd...,1xM6rthhqPRmHfesVPjdCn.mp3,92.253181,1.218599,0.642027,0.332648,3.054225,4.300513,4.660618e-05,6.681142e-05,...,0.001146,0.000141,0.001809,1.310748e-04,0.000676,1.345923e-04,0.000583,0.002017,0.005453,0.001877
2095,audio_chunks/audio.002/1x/1xafrgeBMaBBrnr1e7rk...,1xafrgeBMaBBrnr1e7rkaC.mp3,92.253181,1.853582,0.684296,0.291994,3.000327,4.777125,3.826593e-06,2.911063e-05,...,0.000104,0.000022,0.000178,5.546825e-06,0.000298,1.237812e-05,0.000003,0.000169,0.000718,0.000085
2096,audio_chunks/audio.002/1M/1MTFN8gHszESSnKqXEgh...,1MTFN8gHszESSnKqXEghob.mp3,92.253181,2.013886,0.419342,0.548770,3.728135,3.764989,2.620683e-05,3.597971e-05,...,0.000417,0.000064,0.000605,1.277960e-04,0.000173,1.123576e-04,0.003637,0.001390,0.043248,0.002857
